In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# .py code help to run the code in screen mode .ipynb cannot do that
# This code can be used to load light curve training dataset and do the
# preprocessing like vertical scaling and extending the light curve
# CNN Model is defined
# Training, learning rate scheduler and early stopping feature included

## -- IMP: Check whether the file name to save the model is complete ##

# Import TF and check for GPU

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)

# Import required libraries

import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from keras import layers
from tensorflow.keras.models import save_model, load_model
import math
from sklearn.model_selection import train_test_split
import sys
from numpy import array,append,arange,zeros,exp,sin,random,std
from scipy.interpolate import interp1d

import time


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.15.0


In [ ]:
## Edit the base folder to save model, weights and plots
base_model_folder = '/content/drive/MyDrive/5_AstroFYP_data/april_25_model_training/ml_model/'


# 1. Load Dataset
## Load Train Set
train_shape_dir = '/content/drive/MyDrive/5_AstroFYP_data/april_25_model_training/train_shape_5_2times.npy'
train_lc_dir = '/content/drive/MyDrive/5_AstroFYP_data/april_25_model_training/concat_lc_10_12_shape_5_multisize_multiLDC.npy'

print(f"train_shape_dir = {train_shape_dir}")
print(f"train_lc_dir = {train_lc_dir}")

train_lc_org = np.load(train_lc_dir)
train_shape = np.load(train_shape_dir)
# Check equality of number of dataset
print('train_lc_org.shape =',train_lc_org.shape)
print('train_shape.shape = ',train_shape.shape)
if len(train_lc_org)==len(train_shape):
    print("Train Set: No. of LC = No. of shapes")
else:
    sys.exit("EXIT: Train Set: No. of LC != No. of shapes")

## Load Validation Set
vald_shape_dir = '/content/drive/MyDrive/5_AstroFYP_data/april_25_model_training/vald_shape_1.npy'
vald_lc_dir = '/content/drive/MyDrive/5_AstroFYP_data/april_25_model_training/vald_lc_10_shape_multisize_multiLDC.npy'

print(f"vald_shape_dir = {vald_shape_dir}")
print(f"vald_lc_dir = {vald_lc_dir}")

vald_lc_org = np.load(vald_lc_dir)
vald_shape = np.load(vald_shape_dir)
# Check equality of nuftmber of dataset
print('vald_lc.shape =',vald_lc_org.shape)
print('vald_shape.shape = ',vald_shape.shape)
if len(vald_lc_org)==len(vald_shape):
    print("Vald Set: No. of LC = No. of shapes")
else:
    sys.exit("Vald Set: No. of LC = No. of shapes")

# 2. Normalize the image, convert to opacity map
## Train Set
train_shape = train_shape/np.amax(train_shape)

train_shape[np.where(train_shape == 0)] = 5  # 1 represent the shape (1 opacity)
train_shape[np.where(train_shape == 1)] = 0  # 0 represent background (0 opacity)
train_shape[np.where(train_shape == 5)] = 1

## Valdn Set
vald_shape = vald_shape/np.amax(vald_shape)

vald_shape[np.where(vald_shape == 0)] = 5  # 1 represent the shape (1 opacity)
vald_shape[np.where(vald_shape == 1)] = 0  # 0 represent background (0 opacity)
vald_shape[np.where(vald_shape == 5)] = 1


print("Normalized the shape")



train_shape_dir = /content/drive/MyDrive/5_AstroFYP_data/april_25_model_training/train_shape_5_2times.npy
train_lc_dir = /content/drive/MyDrive/5_AstroFYP_data/april_25_model_training/concat_lc_10_12_shape_5_multisize_multiLDC.npy
train_lc_org.shape = (264272, 100)
train_shape.shape =  (264272, 38, 38)
Train Set: No. of LC = No. of shapes
vald_shape_dir = /content/drive/MyDrive/5_AstroFYP_data/april_25_model_training/vald_shape_1.npy
vald_lc_dir = /content/drive/MyDrive/5_AstroFYP_data/april_25_model_training/vald_lc_10_shape_multisize_multiLDC.npy
vald_lc.shape = (1000, 100)
vald_shape.shape =  (1000, 38, 38)
Vald Set: No. of LC = No. of shapes
Normalized the shape


In [ ]:
train_lc = train_lc_org
vald_lc = vald_lc_org

# 3. Vertically Scaling
## - Train Set

for i in np.arange(len(train_lc)):
    train_lc[i] = (train_lc[i] - np.amin(train_lc[i]))/(np.amax(train_lc[i]) - np.amin(train_lc[i]))

## - Vald Set

for i in np.arange(len(vald_lc)):
    vald_lc[i] = (vald_lc[i] - np.amin(vald_lc[i]))/(np.amax(vald_lc[i]) - np.amin(vald_lc[i]))
print("Vertically Scaled the light curves")


# Add flat line towards left and right of dip
# 10 data points on each side
# 4. Extend the lightcurves
## - Train Set
train_lc_scaled_append = np.ones((train_lc.shape[0],150))
for i in np.arange(len(train_lc)):
    train_lc_scaled_append[i,25:125] = train_lc[i]

del train_lc
train_lc = train_lc_scaled_append
del train_lc_scaled_append

## - Vald Set
vald_lc_scaled_append = np.ones((vald_lc.shape[0],150))
for i in np.arange(len(vald_lc)):
    vald_lc_scaled_append[i,25:125] = vald_lc[i]

del vald_lc
vald_lc = vald_lc_scaled_append
del vald_lc_scaled_append
print("Extended the light curves")


Vertically Scaled the light curves
Extended the light curves


In [ ]:
# 5. Horizontal scaling
def scale_horizontally(input_lc_dataset):
    # lc_np_array_offset_mask used to select the flat part by certain percentage
    input_lc_dataset_mask = np.copy(input_lc_dataset)

    for iteration in np.arange(len(input_lc_dataset)):
        # 0.988 is working good | lower it and see changes # 0.96 - 0.97 -better # 0.95 -worse
        input_lc_dataset_mask[iteration][(input_lc_dataset[iteration]>=0.98)] = 1.0
        input_lc_dataset_mask[iteration][(input_lc_dataset[iteration]<0.98)] = 0.0

    print("Length of one LC = ", len(input_lc_dataset_mask[0]))

    count_zeros_array = np.zeros((len(input_lc_dataset_mask),))
    for iteration in np.arange(len(input_lc_dataset_mask)):
        # Calculate the number of occurrences of '0'
        count_zeros = np.count_nonzero(input_lc_dataset_mask[iteration] == 0)
        count_zeros_array[iteration] = count_zeros

    # Interpolate the light curve
    input_lc_dataset_interpol = np.zeros((len(input_lc_dataset), 120))
    len_selected_portion = np.zeros(len(input_lc_dataset))
    print("input_lc_dataset_interpol.shape =", input_lc_dataset_interpol.shape)

    center_index = int(len(input_lc_dataset[0])/2)
    print("center_index =", center_index)

    for iteration in np.arange(len(input_lc_dataset_interpol)):

        left_index = int(center_index - int(count_zeros_array[iteration]/2) - int(count_zeros_array[iteration]/6))
        right_index = int(center_index + int(count_zeros_array[iteration]/2) + int(count_zeros_array[iteration]/6))
        selected_portion = input_lc_dataset[iteration][left_index:right_index]
        # print("left_index =", left_index)
        # print("right_index =", right_index)

        # Calculate the length of the selected region
        len_selected_portion[iteration] = len(selected_portion)

        # Interpolate the selected portion
        # Original data
        original_x = np.linspace(-1, 1, num=len(selected_portion))
        original_y = selected_portion

        # Create a quadratic interpolation function
        f = interp1d(original_x, original_y, kind='quadratic')

        # Define the range of x-values for the interpolation with 120 elements
        x_interpolation = np.linspace(-1, 1, num=120)

        # Perform the interpolation
        y_interpolated = f(x_interpolation)
        input_lc_dataset_interpol[iteration] = y_interpolated

    return input_lc_dataset_interpol

train_lc = scale_horizontally(train_lc)
vald_lc = scale_horizontally(vald_lc)

print("train_lc.shape = ", train_lc.shape)
print("vald_lc.shape = ", vald_lc.shape)

Length of one LC =  150
input_lc_dataset_interpol.shape = (264272, 120)
center_index = 75
Length of one LC =  150
input_lc_dataset_interpol.shape = (1000, 120)
center_index = 75
train_lc.shape =  (264272, 120)
vald_lc.shape =  (1000, 120)


In [ ]:
# Verification
# Plot - Train LCs
plt.clf()
num = 3
fig,ax=plt.subplots(num,
                    2,
                    figsize=(6,5),
                    gridspec_kw={'width_ratios': [2,1], 'wspace': 0.2, 'hspace': 0.4}
)

ax[0][1].set_title('Shape',size=15)
ax[0][0].set_title('Light Curve (Train Dataset)',size=15)
ax[num-1][0].set_xlabel('Phase',size=13)
ph = np.linspace(-1,1,len(train_lc[0]))

i = 0
for i in np.arange(0, num):
    k = np.random.randint(0, len(train_lc)-1)
    ax[i][1].tick_params(left=False,
                         right=False,
                         labelleft=False,
                         labelbottom=False,
                         bottom=False
    )
    if(i<num-1): ax[i][0].tick_params(labelbottom = False, bottom = False)
    img = ax[i][1].imshow(train_shape[k], cmap='inferno')
    plt.colorbar(img)
    ax[i][0].set_ylabel('Flux', size=13)
    # ax[i][0].set_ylim(-0.5, 1.5)
    ax[i][0].plot(ph, train_lc[k], color = 'tab:red', linewidth='2')
    ax[i][0].grid('on')
    i = i + 1
plt.savefig(base_model_folder+'plot_train_lc.png')
plt.close()

# Plot - Vald LCs
plt.clf()
num = 3
fig,ax=plt.subplots(num,
                    2,
                    figsize=(4, 3),
                    gridspec_kw={'width_ratios':[2,1], 'wspace':0.2, 'hspace':0.4})

ax[0][1].set_title('Shape',size=15)
ax[0][0].set_title('Light Curve (Vald Dataset)',size=15)
ax[num-1][0].set_xlabel('Phase',size=13)
ph = np.linspace(-1,1,len(vald_lc[0]))

i = 0
for i in np.arange(0,num):
    k = np.random.randint(0, len(vald_lc)-1)
    ax[i][1].tick_params(left=False,
                         right=False,
                         labelleft=False,
                         labelbottom=False,
                         bottom=False
    )
    if(i<num-1): ax[i][0].tick_params(labelbottom = False, bottom = False)
    img = ax[i][1].imshow(vald_shape[k],cmap='inferno')
    plt.colorbar(img)
    ax[i][0].set_ylabel('Flux',size=13)
    # ax[i][0].set_ylim(-0.5,1.5)
    # ax[i][0].scatter(ph, vald_lc_scaled_append[k],color = 'black',marker='.')
    ax[i][0].plot(ph, vald_lc[k], color = 'tab:red', linewidth='2')
    ax[i][0].grid('on')
    i = i + 1
plt.savefig(base_model_folder+'plot_vald_lc.png')
plt.close()

<Figure size 640x480 with 0 Axes>

In [ ]:
## START - Add noise
random_generator = np.random.default_rng()

SNR_array_train = random_generator.uniform(50, 500, len(train_lc))
std_dev_train = 1/SNR_array_train
del SNR_array_train

SNR_array_vald = random_generator.uniform(50, 500, len(vald_lc))
std_dev_vald = 1/SNR_array_vald
del SNR_array_vald

for i in np.arange(len(train_lc)):
    train_lc[i] = train_lc[i] + np.random.normal(loc=0.0, scale=std_dev_train[i], size=len(train_lc[i]))
del std_dev_train

for i in np.arange(len(vald_lc)):
    vald_lc[i] = vald_lc[i] + np.random.normal(loc=0.0, scale=std_dev_vald[i], size=len(vald_lc[i]))
del std_dev_vald
## END - Add noise
print("After adding noise")

print(f"train_lc.shape = {train_lc.shape}")
print(f"vald_lc.shape = {vald_lc.shape}")

After adding noise
train_lc.shape = (264272, 120)
vald_lc.shape = (1000, 120)


In [ ]:
# Verification
# Plot - Train LCs
plt.clf()
num = 3
fig,ax=plt.subplots(num,
                    2,
                    figsize=(6,5),
                    gridspec_kw={'width_ratios': [2,1], 'wspace': 0.2, 'hspace': 0.4}
)

ax[0][1].set_title('Shape',size=15)
ax[0][0].set_title('Light Curve (Train Dataset)',size=15)
ax[num-1][0].set_xlabel('Phase',size=13)
ph = np.linspace(-1,1,len(train_lc[0]))

i = 0
for i in np.arange(0, num):
    k = np.random.randint(0, len(train_lc)-1)
    ax[i][1].tick_params(left=False,
                         right=False,
                         labelleft=False,
                         labelbottom=False,
                         bottom=False
    )
    if(i<num-1): ax[i][0].tick_params(labelbottom = False, bottom = False)
    img = ax[i][1].imshow(train_shape[k], cmap='inferno')
    plt.colorbar(img)
    ax[i][0].set_ylabel('Flux', size=13)
    # ax[i][0].set_ylim(-0.5, 1.5)
    ax[i][0].plot(ph, train_lc[k], color = 'tab:red', linewidth='2')
    ax[i][0].grid('on')
    i = i + 1
plt.savefig(base_model_folder+'plot_train_lc_noise.png')
plt.close()

# Plot - Vald LCs
plt.clf()
num = 3
fig,ax=plt.subplots(num,
                    2,
                    figsize=(4, 3),
                    gridspec_kw={'width_ratios':[2,1], 'wspace':0.2, 'hspace':0.4})

ax[0][1].set_title('Shape',size=15)
ax[0][0].set_title('Light Curve (Vald Dataset)',size=15)
ax[num-1][0].set_xlabel('Phase',size=13)
ph = np.linspace(-1,1,len(vald_lc[0]))

i = 0
for i in np.arange(0,num):
    k = np.random.randint(0, len(vald_lc)-1)
    ax[i][1].tick_params(left=False,
                         right=False,
                         labelleft=False,
                         labelbottom=False,
                         bottom=False
    )
    if(i<num-1): ax[i][0].tick_params(labelbottom = False, bottom = False)
    img = ax[i][1].imshow(vald_shape[k],cmap='inferno')
    plt.colorbar(img)
    ax[i][0].set_ylabel('Flux',size=13)
    # ax[i][0].set_ylim(-0.5,1.5)
    # ax[i][0].scatter(ph, vald_lc_scaled_append[k],color = 'black',marker='.')
    ax[i][0].plot(ph, vald_lc[k], color = 'tab:red', linewidth='2')
    ax[i][0].grid('on')
    i = i + 1
plt.savefig(base_model_folder+'plot_vald_lc_noise.png')
plt.close()


# Model path to be saved to
model_save_path = base_model_folder+"april24_2024_model_unfDist_LDC_size_horz_scale.h5"
print(f"model_save_path = {model_save_path}")

no_epochs = int(200) # For testing start with small value of 3 or 5
print("no_epochs =",no_epochs)

# user_input = input("Do you want to run the code? (y/n): ")
# if user_input.lower() != "y":
#     sys.exit("EXIT: User declined to continue the program")






model_save_path = /content/drive/MyDrive/5_AstroFYP_data/april_25_model_training/ml_model/april24_2024_model_unfDist_LDC_size_horz_scale.h5
no_epochs = 200


<Figure size 640x480 with 0 Axes>

In [ ]:
tic = time.perf_counter()
print("Creating ML Pipeline")
# 6. ML Pipeline
## Train Set
train_dataset = tf.data.Dataset.from_tensor_slices((train_lc, train_shape))
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(len(train_dataset))
train_dataset = train_dataset.batch(100)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
print(train_dataset)

## Vald Set
vald_dataset = tf.data.Dataset.from_tensor_slices((vald_lc, vald_shape))
vald_dataset = vald_dataset.batch(100)
vald_dataset = vald_dataset.cache()
vald_dataset = vald_dataset.prefetch(tf.data.AUTOTUNE)
print(vald_dataset)

Creating ML Pipeline
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 120), dtype=tf.float64, name=None), TensorSpec(shape=(None, 38, 38), dtype=tf.float64, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 120), dtype=tf.float64, name=None), TensorSpec(shape=(None, 38, 38), dtype=tf.float64, name=None))>


In [ ]:
# CNN Model
input_shape = np.array(np.shape(train_lc[0]))
print("np.shape(input_shape) =", input_shape[0])

output_shape = np.array(np.shape(train_shape[0]))
print("np.shape(input_shape) =", output_shape[0], output_shape[1])

START = input_shape[0]
END = output_shape[0]
print("Start =", START)
print("End =", END)

conv_ip = keras.layers.Input(shape=(START,), name='Input')
x= keras.layers.Reshape((START, 1), input_shape=(START,), name='reshape_1')(conv_ip)
x= keras.layers.BatchNormalization()(x)

x=keras.layers.Conv1D(16,
                      kernel_size=5,
                      strides=1,
                      activation='relu',
                      name='conv16_5',
                      padding='same')(x)

x=keras.layers.Conv1D(16,
                      kernel_size=5,
                      strides=1,
                      activation='relu',
                      name='second_conv16_5',
                      padding='same')(x)

x=keras.layers.MaxPool1D(5,
                         strides=2,
                         data_format='channels_last',
                         name='maxpool_1',
                         padding='same')(x)

x=keras.layers.Conv1D(32,
                      kernel_size=5,
                      strides=1,
                      activation='relu',
                      name='first_conv32_5',
                      padding='same')(x)

x=keras.layers.Conv1D(32,
                      kernel_size=5,
                      strides=1,
                      activation='relu',
                      name='second_conv32_5',
                      padding='same')(x)

x=keras.layers.MaxPool1D(5,
                         strides=2,
                         data_format='channels_last',
                         name='maxpool_2',
                         padding='same')(x) #200

x=keras.layers.Conv1D(64,
                      kernel_size=5,
                      strides=1,
                      activation='relu',
                      name='first_conv64_5',
                      padding='same')(x)

x=keras.layers.Conv1D(64,
                      kernel_size=5,
                      strides=1,
                      activation='relu',
                      name='second_conv64_5',
                      padding='same')(x)

x=keras.layers.MaxPool1D(5,
                         strides=2,
                         data_format='channels_last',
                         name='maxpool_3',
                         padding='same')(x)

x=keras.layers.Flatten(name='flat_1')(x)

x=keras.layers.Dense(256, name='dense_layer_5', activation='relu')(x)
x=keras.layers.Dense(256, name='dense_layer_6', activation='relu')(x)

x= keras.layers.Dense(END**2, name='dense_layer_u', activation='relu')(x)
x = keras.layers.Reshape(target_shape=(END, END, 1), name='reshape_2')(x)

x=keras.layers.Conv2D(32,
                       kernel_size=(3,3),
                       strides=1,
                       activation='relu',
                       name='second_conv64_52',
                       padding='same')(x)

x=keras.layers.Conv2D(32,
                       kernel_size=(3,3),
                       strides=1,
                       activation='relu',
                       name='second_conv64_522',
                       padding='same')(x)

x=keras.layers.Conv2D(16,
                       kernel_size=(3,3),
                       strides=1,
                       activation='relu',
                       name='second_conv64_524',
                       padding='same')(x)

x=keras.layers.Conv2D(1,
                       kernel_size=3,
                       strides=1,
                       activation='relu',
                       name='second_conv64_53',
                       padding='same')(x)

conv_op = keras.layers.Reshape(target_shape=(END, END), name='reshape_3')(x)

model = keras.Model(inputs=conv_ip, outputs=conv_op, name="predict_shape_from_LC")
model.summary()

print("Model is defined")

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='mse')
print("Model is compiled")

# Patience early stopping
es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                   mode='min',
                                   verbose=1,
                                   patience=30
)
print("Early stopping defined")

# Learning rate scheduler
def step_decay(epoch):
	initial_lrate = 0.001
	drop = 0.5
	epochs_drop = 20
	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lrate
lr_sched = keras.callbacks.LearningRateScheduler(step_decay)
print("Learning rate scheduler defined")

# Model checkpoint
checkpoint_path = base_model_folder+"ckpt/checkpoint.weights.h5"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min',
    save_freq='epoch',
    initial_value_threshold=None
)
print("Model checkpoint defined")



np.shape(input_shape) = 120
np.shape(input_shape) = 38 38
Start = 120
End = 38
Model: "predict_shape_from_LC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 120)]             0         
                                                                 
 reshape_1 (Reshape)         (None, 120, 1)            0         
                                                                 
 batch_normalization (Batch  (None, 120, 1)            4         
 Normalization)                                                  
                                                                 
 conv16_5 (Conv1D)           (None, 120, 16)           96        
                                                                 
 second_conv16_5 (Conv1D)    (None, 120, 16)           1296      
                                                                 
 maxpool_1 (MaxPooling1D)    (No

In [ ]:
# Training
print("Training will start now")

history = model.fit(train_dataset,
                    epochs=no_epochs,
                    verbose=2,
                    validation_data=vald_dataset,
                    callbacks=[es, lr_sched, model_checkpoint_callback]
)

# Save Model
save_model(model, str(model_save_path))
print("Model saved")

plt.plot(history.history['loss'], label="Training Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.legend()
plt.savefig(base_model_folder+'history_loss_graph.png')
plt.close()

toc = time.perf_counter()
print(toc-tic, " s")



Training will start now
Epoch 1/200
2643/2643 - 44s - loss: 0.1578 - val_loss: 0.1494 - lr: 0.0010 - 44s/epoch - 17ms/step
Epoch 2/200
2643/2643 - 32s - loss: 0.1491 - val_loss: 0.1485 - lr: 0.0010 - 32s/epoch - 12ms/step
Epoch 3/200
2643/2643 - 32s - loss: 0.1481 - val_loss: 0.1472 - lr: 0.0010 - 32s/epoch - 12ms/step
Epoch 4/200
2643/2643 - 31s - loss: 0.1475 - val_loss: 0.1463 - lr: 0.0010 - 31s/epoch - 12ms/step
Epoch 5/200
2643/2643 - 31s - loss: 0.1470 - val_loss: 0.1466 - lr: 0.0010 - 31s/epoch - 12ms/step
Epoch 6/200
2643/2643 - 31s - loss: 0.1466 - val_loss: 0.1456 - lr: 0.0010 - 31s/epoch - 12ms/step
Epoch 7/200
2643/2643 - 31s - loss: 0.1462 - val_loss: 0.1457 - lr: 0.0010 - 31s/epoch - 12ms/step
Epoch 8/200
2643/2643 - 31s - loss: 0.1459 - val_loss: 0.1450 - lr: 0.0010 - 31s/epoch - 12ms/step
Epoch 9/200
2643/2643 - 31s - loss: 0.1456 - val_loss: 0.1453 - lr: 0.0010 - 31s/epoch - 12ms/step
Epoch 10/200
2643/2643 - 31s - loss: 0.1454 - val_loss: 0.1452 - lr: 0.0010 - 31s/epo

<ipython-input-11-fbd7d9062525>:12: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, str(model_save_path))


Model saved
3860.5747726  s


In [ ]:
# save_model(model, str(model_save_path))


In [ ]:
# tf.keras.backend.clear_session()
# print("End of code")